In [2]:
import os 

In [3]:
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper


In [4]:
## used inbuilt wikipedia tools

api_wiki_tools=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wiki_tools)
wiki.name

'wikipedia'

In [5]:
api_arvi_tools=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
ariv=ArxivQueryRun(api_wrapper=api_arvi_tools)
ariv.name

'arxiv'

In [6]:
tools=[wiki,ariv]


In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
embedding = OllamaEmbeddings(model="mxbai-embed-large")
loader=WebBaseLoader("https://docs.smith.langchain.com/")
data=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
text_splitter





In [9]:
split_doct=text_splitter.split_documents(data)
split_doct

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'LangSmith docs - Docs by LangChain', 'language': 'en'}, page_content='LangSmith docs - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewCreate an account and API keyIntegrationsPlansAccount administrationOverviewSet up hierarchyWorkload isolationManage organizations using the APIManage billingGranular usageSet up resource tagsUser managementAdditional resourcesPolly AI assistantBetaData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusLangSmith docsCopy pageCopy pageLangSmith is a framework-agnostic platform for developing, debugging, and deploying AI agents and LLM applications.'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 

In [10]:
vector_store = FAISS.from_documents(split_doct, embedding)


In [11]:
retiever=vector_store.as_retriever()
retiever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026FEB4DA3E0>)

In [12]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=retiever,  
    name="langsmith_search", 
    description="Search any information about LangSmith"
)

retriever_tool

Tool(name='langsmith_search', description='Search any information about LangSmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000026FF3202E60>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026FEB4DA3E0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000026FF3202EF0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026FEB4DA3E0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))

In [13]:
tools=[wiki,ariv,retriever_tool]
tools


[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\kakab\\OneDrive\\Desktop\\GEN AI\\Q&A Project\\project_1\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search any information about LangSmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000026FF3202E60>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS o

In [14]:
import os 
from dotenv import load_dotenv

load_dotenv()
groq_api_key= os.getenv("GROQ_API_KEY")


In [15]:
import os
import certifi

os.environ["SSL_CERT_FILE"] = certifi.where()


In [16]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model="llama-3.1-8b-instant"
  
)

In [17]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages



[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [18]:
from langchain.agents import create_openai_tools_agent,AgentExecutor

agent=create_openai_tools_agent(llm,tools,prompt)

In [19]:
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)


In [20]:
agent_executor.invoke({"input":"whats is llm"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `langsmith_search` with `{'query': 'llm definition'}`


LangSmith docs - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewCreate an account and API keyIntegrationsPlansAccount administrationOverviewSet up hierarchyWorkload isolationManage organizations using the APIManage billingGranular usageSet up resource tagsUser managementAdditional resourcesPolly AI assistantBetaData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusLangSmith docsCopy pageCopy pageLangSmith is a framework-agnostic platform for developing, debugging, and deploying AI agents and LLM applications.

It helps you trace requests, evaluate outputs, test prompts, and manage deployments in one place. LangSmith works with any agent stack, whether you’re using a

{'input': 'whats is llm',
 'output': 'Here are some real-world applications of LLMs in various industries:\n\n1.  **Customer Service**: LLMs can be used to create chatbots that can understand and respond to customer inquiries, freeing up human customer support agents to focus on more complex issues.\n2.  **Content Generation**: LLMs can generate high-quality content, such as articles, social media posts, and product descriptions, saving time and resources for content creators.\n3.  **Language Translation**: LLMs can translate languages in real-time, enabling businesses to communicate with customers and partners across language barriers.\n4.  **Virtual Assistants**: LLMs can be integrated into virtual assistants like Siri, Alexa, and Google Assistant to provide more accurate and helpful responses to user queries.\n5.  **Marketing and Advertising**: LLMs can help create personalized marketing campaigns, generate ad copy, and analyze customer behavior to improve marketing ROI.\n6.  **Heal

In [21]:
agent_executor.invoke({"input":"whats is  machine learning"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `langsmith_search` with `{'query': 'machine learning'}`


It helps you trace requests, evaluate outputs, test prompts, and manage deployments in one place. LangSmith works with any agent stack, whether you’re using an existing framework or building your own. Prototype locally, then move to production with integrated monitoring and evaluation to build more reliable AI agents.
​Get started
Create an accountSign up at smith.langchain.com (no credit card required).
You can log in with Google, GitHub, or email.Create an API keyGo to your Settings page → API Keys → Create API Key.
Copy the key and save it securely.Choose your integrationLangSmith works with many frameworks a

KeyboardInterrupt: 